Ön hazırlık (kütüphane kurulumu)

In [4]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

SHOR'UN ALGORİTMASI

Cirq ile Shor Algoritması (Basitleştirilmiş)

In [5]:
import cirq
import numpy as np

# 4 qubitlik bir sistem (basit örnek)
qubits = [cirq.LineQubit(i) for i in range(4)]

# Başlangıç devresi
circuit = cirq.Circuit()

# Hadamard kapılarıyla süperpozisyon
circuit.append([cirq.H(q) for q in qubits])

# Basitleştirilmiş modüler üstel operatör (örnek için)
# Bu örnekte doğrudan modüler çarpma yerine X kapıları kullanıyoruz
circuit.append(cirq.CNOT(qubits[0], qubits[1]))
circuit.append(cirq.CNOT(qubits[1], qubits[2]))
circuit.append(cirq.CNOT(qubits[2], qubits[3]))

# Ölçüm
circuit.append([cirq.measure(q) for q in qubits])

# Simülasyon
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)
print(result)


q(0)=0111111111
q(1)=0001111001
q(2)=0000000100
q(3)=0100110111


Qiskit ile Shor Algoritması (Basitleştirilmiş)

In [6]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

# 4 qubitlik bir devre
qc = QuantumCircuit(4, 4)

# Hadamard ile süperpozisyon
for i in range(4):
    qc.h(i)

# Basitleştirilmiş modüler üstel (X kapılarıyla örnek)
qc.cx(0,1)
qc.cx(1,2)
qc.cx(2,3)

# Ölçüm
qc.measure(range(4), range(4))

# Simülasyon
backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=10).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Basitleştirilmiş Shor Algoritması

In [7]:
import numpy as np
from qutip import *

# Qubit sayısı (basit örnek)
n_qubits = 4

# Başlangıç durumu |0000>
psi0 = tensor([basis(2,0) for _ in range(n_qubits)])

# Hadamard ile süperpozisyon: H ⊗ H ⊗ H ⊗ H
H = hadamard_transform(n_qubits)
psi1 = H * psi0

# Basitleştirilmiş modüler üstel operatör
# Örnek: a^x mod 15 fonksiyonu yerine basit bir permutation (demo)
# Gerçek Shor'da burada modüler üstel operatör uygulanır
U_mod = qeye(2**n_qubits)
U_mod[1,1] = 0
U_mod[1,2] = 1
U_mod[2,2] = 0
U_mod[2,1] = 1

psi2 = U_mod * psi1

# Ölçüm simülasyonu
# Qutip ölçüm projeksiyonları
proj0 = tensor([basis(2,0)*basis(2,0).dag() for _ in range(n_qubits)])
proj1 = qeye(2**n_qubits) - proj0

# Ölçüm sonuçlarını hesapla
prob0 = (psi2.dag() * proj0 * psi2).full()[0,0].real
prob1 = (psi2.dag() * proj1 * psi2).full()[0,0].real

print("Ölçüm olasılığı |0000>: {:.2f}".format(prob0))
print("Ölçüm olasılığı diğer durumlar: {:.2f}".format(prob1))


NameError: name 'hadamard_transform' is not defined

QUANTUM PHASE ESTIMATION (QPE)

Cirq ile QPE Örneği

In [10]:
import cirq
import numpy as np

# ------------------------------
# Parametreler
# ------------------------------
n_control = 3
phi = 1/4

# Qubitler
controls = [cirq.GridQubit(0, i) for i in range(n_control)]
target = cirq.GridQubit(1, 0)

# Devre
circuit = cirq.Circuit()

# Başlangıç: kontrol qubitleri H ile süperpozisyon
circuit.append([cirq.H(q) for q in controls])

# Hedef qubit hazırla |1> (U özvektörü için)
circuit.append(cirq.X(target))

# Kontrollü-U^2^j işlemleri
for j, q in enumerate(controls):
    exponent = 2**j * phi
    circuit.append(cirq.CZPowGate(exponent=exponent*2).on(q, target))

# Inverse QFT
def qft_rotations(qubits):
    if len(qubits) == 0:
        return []
    q = qubits[0]
    rest = qubits[1:]
    ops = qft_rotations(rest)
    for i, r in enumerate(rest):
        ops.append(cirq.CZPowGate(exponent=-1/2**(i+1)).on(r,q))
    ops.append(cirq.H(q))
    return ops

circuit.append(qft_rotations(controls))
circuit.append(cirq.measure(*controls, key='result'))

# Simülasyon
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)

print("Cirq QPE Ölçüm Sonuçları:")
print(result.histogram(key='result'))


Cirq QPE Ölçüm Sonuçları:
Counter({2: 1000})


Qiskit ile QPE Örneği

In [9]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import QFT

# ------------------------------
# Parametreler
# ------------------------------
n_control = 3  # Kontrol qubit sayısı
phi = 1/4      # Hedef faz
U = [[1, 0], [0, complex(np.exp(2*np.pi*1j*phi))]]  # 1-qubit üniter

# ------------------------------
# QPE Devresi
# ------------------------------
qc = QuantumCircuit(n_control+1, n_control)

# Hadamard Kapıları (kontrol register)
for q in range(n_control):
    qc.h(q)

# Kontrollü-U operatörleri
for j in range(n_control):
    power = 2**j
    qc.cu1(2*np.pi*phi*power, j, n_control)

# Inverse QFT
qc.append(QFT(num_qubits=n_control, do_swaps=True).inverse(), range(n_control))

# Ölçüm
qc.measure(range(n_co_


SyntaxError: incomplete input (ipython-input-560160484.py, line 29)

QuTiP ile Basit QPE Örneği

In [8]:
import numpy as np
from qutip import *

# ------------------------------
# 1. Parametreler
# ------------------------------
n_control = 2  # Kontrol qubit sayısı
n_target = 1   # Hedef qubit sayısı

# Hedef üniter operatör U ve özvektör |u> (Hedef qubit)
phi = 0.25  # Faz = 1/4
U = Qobj([[1, 0], [0, np.exp(2*np.pi*1j*phi)]])
u = basis(2,0)  # |0> özvektör

# Başlangıç durumu: |0...0> ⊗ |u>
psi0 = tensor([basis(2,0) for _ in range(n_control)] + [u])

# ------------------------------
# 2. Hadamard Kapıları
# ------------------------------
H = hadamard_transform(n_control)
H_full = tensor(H, qeye(2**n_target))
psi1 = H_full * psi0

# ------------------------------
# 3. Kontrollü-U Operatörleri
# ------------------------------
def controlled_U(U, control_index, n_total):
    # Basit 2-qubit kontrol için
    P0 = tensor([basis(2,0)*basis(2,0).dag() if i==control_index else qeye(2) for i in range(n_total)])
    P1 = tensor([basis(2,1)*basis(2,1).dag() if i==control_index else qeye(2) for i in range(n_total)])
    U_ext = tensor([qeye(2) for _ in range(n_total-1)] + [U])
    return P0 + P1 * U_ext

# n_control = 2 için U, U^2 uygula
psi2 = psi1
for j in range(n_control):
    Uj = U**(2**j)
    CU = controlled_U(Uj, j, n_control+n_target)
    psi2 = CU * psi2

# ------------------------------
# 4. Inverse QFT (basit 2 qubit)
# ------------------------------
def inverse_qft(psi, n):
    """2 qubit için basit Inverse QFT"""
    psi = swap(psi, 0, 1)  # Swap qubit 0 ve 1
    psi = qft([psi], dagger=True)  # Qutip qft fonksiyonu
    return psi

# Burada basitleştirilmiş gösterim
# psi3 = inverse_qft(psi2, n_control)  # 2 qubit register

# ------------------------------
# 5. Ölçüm (kontrol register)
# ------------------------------
# Not: QuTiP ölçüm fonksiyonları ile kontrol register olasılıkları bulunabilir
proj_list = [tensor([basis(2,i)*basis(2,i).dag() for _ in range(n_control)] + [qeye(2)]) for i in range(2**n_control)]
probs = [abs((proj * psi2).norm())**2 for proj in proj_list]

print("Kontrol register olasılıkları:")
for i, p in enumerate(probs):
    print(f"{i:02b}: {p:.2f}")


NameError: name 'hadamard_transform' is not defined

QuTiP ile QPE Örneği (Geliştirilmiş)

In [11]:
import numpy as np
from qutip import *

# ------------------------------
# Parametreler
# ------------------------------
n_control = 3
phi = 1/4
n_total = n_control + 1  # toplam qubit sayısı

# Hedef özvektör |u> ve üniter U
u = basis(2,0)
U = Qobj([[1,0],[0,np.exp(2*np.pi*1j*phi)]])

# Başlangıç durumu
psi0 = tensor([basis(2,0) for _ in range(n_control)] + [u])

# Hadamard Kapıları
H_list = [hadamard_transform(1) for _ in range(n_control)]
H_full = tensor(H_list + [qeye(2)])
psi1 = H_full * psi0

# Kontrollü-U işlemleri
def controlled_U(U, ctrl, tgt, n_total):
    P0 = tensor([basis(2,0)*basis(2,0).dag() if i==ctrl else qeye(2) for i in range(n_total)])
    P1 = tensor([basis(2,1)*basis(2,1).dag() if i==ctrl else qeye(2) for i in range(n_total)])
    U_ext = tensor([qeye(2) if i!=tgt else U for i in range(n_total)])
    return P0 + P1*U_ext

psi2 = psi1
for j in range(n_control):
    psi2 = controlled_U(U**(2**j), j, n_control, n_total) * psi2

# Basit ölçüm: kontrol register olasılıkları
probs = []
for i in range(2**n_control):
    proj = tensor([basis(2,(i>>j)&1)*basis(2,(i>>j)&1).dag() for j in range(n_control)] + [qeye(2)])
    probs.append(abs((proj * psi2).norm())**2)

print("QuTiP QPE Ölçüm Olasılıkları:")
for i, p in enumerate(probs):
    print(f"{i:03b}: {p:.3f}")


NameError: name 'hadamard_transform' is not defined

ORDER FINDING ALGORİTMASI

Cirq ile Order Finding Örneği

In [12]:
import cirq
import numpy as np

# ------------------------------
# Parametreler
# ------------------------------
n_control = 4
N = 15
a = 2

# Qubitler
controls = [cirq.GridQubit(0, i) for i in range(n_control)]
targets = [cirq.GridQubit(1, i) for i in range(4)]  # hedef register

# Devre
circuit = cirq.Circuit()

# Başlangıç: kontrol qubitleri H ile süperpozisyon
circuit.append([cirq.H(q) for q in controls])

# Hedef register |1>
circuit.append(cirq.X(targets[0]))

# ------------------------------
# Basitleştirilmiş kontrollü modüler üslü operatör
# ------------------------------
# Gerçek U_a^2^j operatörü için kuantum mantık kapıları gerekir
for j, q in enumerate(controls):
    for t in targets:
        circuit.append(cirq.CX(q, t))  # örnek amaçlı, küçük simülasyon

# Inverse QFT
def qft_rotations(qubits):
    if len(qubits) == 0:
        return []
    q = qubits[0]
    rest = qubits[1:]
    ops = qft_rotations(rest)
    for i, r in enumerate(rest):
        ops.append(cirq.CZPowGate(exponent=-1/2**(i+1)).on(r,q))
    ops.append(cirq.H(q))
    return ops

circuit.append(qft_rotations(controls))
circuit.append(cirq.measure(*controls, key='result'))

# Simülasyon
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)

print("Cirq Order Finding Ölçüm Sonuçları:")
print(result.histogram(key='result'))


Cirq Order Finding Ölçüm Sonuçları:
Counter({0: 515, 5: 146, 11: 121, 3: 75, 13: 59, 9: 44, 7: 35, 1: 5})


Qiskit ile Order Finding Örneği

In [13]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import QFT

# ------------------------------
# Parametreler
# ------------------------------
n_control = 4  # kontrol qubit sayısı (üst register)
N = 15
a = 2

# ------------------------------
# Devre oluşturma
# ------------------------------
qc = QuantumCircuit(n_control+4, n_control)  # 4 qubit hedef register (N<16)
# Başlangıç: kontrol register H kapıları
for q in range(n_control):
    qc.h(q)

# Hedef register |1> başlangıç
qc.x(n_control)

# ------------------------------
# Kontrollü modüler üs alma (basitleştirilmiş örnek)
# ------------------------------
# Bu kısım gerçek devrede kompleks modüler üslü operatör gerektirir
# Küçük örnek için klasik modüler hesaplama ve controlled-X ile simüle edelim
# Kontrollü-U^2^j işlemleri
for j in range(n_control):
    exponent = 2**j
    # Basit modüler işlemi simüle et (örnek amaçlı)
    for _ in range(exponent):
        for k in range(n_control, n_control+4):
            qc.cx(j, k)  # bu sadece örnek, gerçek U_a^2^j operatörü daha karmaşıktır

# Inverse QFT
qc.append(QFT(num_qubits=n_control, do_swaps=True).inverse(), range(n_control))

# Ölçüm
qc.measure(range(n_control), range(n_control))

# ------------------------------
# Simülasyon
# ------------------------------
backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1024).result()
counts = result.get_counts()

print("Qiskit Order Finding Ölçüm Sonuçları:")
print(counts)
qc.draw('mpl')


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Order Finding (Küçük Sistem Örneği)

In [14]:
import numpy as np
from qutip import basis, tensor, qft, Qobj, measurement_statistics_povm

# ------------------------------
# Parametreler
# ------------------------------
N = 15
a = 2
n_control = 3  # kontrol register (süperpozisyon) qubit sayısı
n_target = 4   # hedef register (N<16 için 4 qubit)

# ------------------------------
# Qubit register başlangıç
# ------------------------------
# Kontrol register |0>
control = tensor([basis(2,0) for _ in range(n_control)])
# Hedef register |1>
target = tensor([basis(2,1)]+[basis(2,0) for _ in range(n_target-1)])
# Başlangıç durumu
psi0 = tensor(control, target)

# ------------------------------
# Hadamard kontrol qubitleri
# ------------------------------
H = (1/np.sqrt(2)) * np.array([[1,1],[1,-1]])
H_control = tensor([Qobj(H) for _ in range(n_control)])
# Süperpozisyon durumu
psi1 = H_control * control
psi1 = tensor(psi1, target)

# ------------------------------
# Basitleştirilmiş kontrollü modüler üs alma
# ------------------------------
# Bu küçük örnek için hedef register mod N üzerindeki döngüsel permütasyon ile simüle edilir
def controlled_modular_exponentiation(psi, a, N, n_control, n_target):
    """
    Küçük örnek: hedef register'da basit permütasyon
    """
    # Bu örnekte modüler üs alma tam simülasyonu yapılmamıştır
    # Ama mantığı göstermek için döngü eklenebilir
    return psi  # basit placeholder

psi2 = controlled_modular_exponentiation(psi1, a, N, n_control, n_target)

# ------------------------------
# Inverse QFT
# ------------------------------
# Sadece kontrol register üzerine uygulayacağız
qft_dagger_control = qft(n_control).dag()
psi3 = tensor(qft_dagger_control * psi2.ptrace(list(range(n_control))), psi2.ptrace(list(range(n_control, n_control+n_target))))

# ------------------------------
# Ölçüm (kontrol register)
# ------------------------------
# Basitleştirilmiş ölçüm: eigenstate prob. simülasyonu
measurement_operators = [basis(2**n_control, i) * basis(2**n_control, i).dag() for i in range(2**n_control)]
probs = [abs((op * psi3.ptrace(list(range(n_control)))).norm())**2 for op in measurement_operators]

print("Kontrol register ölçüm olasılıkları:")
for i, p in enumerate(probs):
    print(f"|{i:0{n_control}b}> : {p:.4f}")


ImportError: cannot import name 'measurement_statistics_povm' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

DISCRETE LOGARİTHM ALGORİTMASI (AYRIK
LOGARİTMA ALGORİTMASI) .

Qiskit Örneği

In [15]:
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.circuit.library import QFT
import numpy as np

# Basitleştirilmiş QFT tabanlı ayrık logaritma simülasyonu
n = 5  # kontrol register boyutu (2^5 > p)
qc = QuantumCircuit(n, n)

# Adım 1: Hadamard ile süperpozisyon
qc.h(range(n))

# Adım 2: Kontrollü modüler üs alma işlemi (küçük örnek için manuel)
# Gerçek Shor tabanlı DLP devresi çok karmaşık, simülasyon için placeholder

# Adım 3: QFT^\dagger
qc.append(QFT(num_qubits=n, do_swaps=False).inverse(), range(n))

# Adım 4: Ölçüm
qc.measure(range(n), range(n))

# Çalıştırma
sim = Aer.get_backend('aer_simulator')
tqc = transpile(qc, sim)
qobj = assemble(tqc)
result = sim.run(qobj).result()
counts = result.get_counts()
print("Qiskit ölçüm sonuçları:", counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

Cirq Örneği

In [16]:
import cirq
import numpy as np

n = 5  # kontrol register boyutu
qubits = [cirq.LineQubit(i) for i in range(n)]

circuit = cirq.Circuit()

# Adım 1: Hadamard ile süperpozisyon
for q in qubits:
    circuit.append(cirq.H(q))

# Adım 2: Kontrollü modüler üs alma (placeholder)
# Gerçek DLP devresi çok karmaşık

# Adım 3: Inverse QFT
def inverse_qft(qubits):
    n = len(qubits)
    for i in range(n//2):
        circuit.append(cirq.SWAP(qubits[i], qubits[n-i-1]))
    for j in range(n):
        for m in range(j):
            circuit.append(cirq.CZ(qubits[j], qubits[m])**- (1/2**(j-m)))
        circuit.append(cirq.H(qubits[j]))

inverse_qft(qubits)

# Adım 4: Ölçüm
circuit.append(cirq.measure(*qubits, key='result'))

# Simülasyon
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)
print("Cirq ölçüm sonuçları:\n", result)


Cirq ölçüm sonuçları:
 result=0000000000, 0000000000, 0000000000, 0000000000, 0000000000


QuTiP Örneği

In [17]:
import numpy as np
from qutip import basis, tensor, qft, Qobj

# Parametreler
n_control = 5
n_target = 5  # küçük örnek
p = 23
g = 5
y = 8

# Başlangıç register
control = tensor([basis(2,0) for _ in range(n_control)])
target = tensor([basis(2,1) for _ in range(n_target)])
psi0 = tensor(control, target)

# Hadamard süperpozisyon
H = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])
H_control = tensor([Qobj(H) for _ in range(n_control)])
psi1 = tensor(H_control*control, target)

# Kontrollü modüler üs alma placeholder
psi2 = psi1  # Küçük örnek simülasyonu için

# Inverse QFT
qft_dagger_control = qft(n_control).dag()
psi3 = tensor(qft_dagger_control*psi2.ptrace(range(n_control)), psi2.ptrace(range(n_control, n_control+n_target)))

# Ölçüm olasılıkları
measurement_operators = [basis(2**n_control,i)*basis(2**n_control,i).dag() for i in range(2**n_control)]
probs = [abs((op*psi3.ptrace(range(n_control))).norm())**2 for op in measurement_operators]

for i, p_ in enumerate(probs):
    if p_>0.01:
        print(f"|{i:0{n_control}b}> : {p_:.4f}")


TypeError: incompatible dimensions [[5], [5]] and [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]]